In [1]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from itertools import product
!pip install kmodes

In [2]:
variables=pd.read_csv('DATA/transport_accidents_2017_18.csv')
variables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93285 entries, 0 to 93284
Data columns (total 58 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   accident_type                          93285 non-null  int64 
 1   gender                                 93285 non-null  int64 
 2   nationality                            93285 non-null  int64 
 3   work_situation                         93285 non-null  int64 
 4   main_occupation                        93285 non-null  int64 
 5   seniority_in_the_job_months            93285 non-null  int64 
 6   seniority_in_the_job_days              93285 non-null  int64 
 7   contract_type                          93285 non-null  int64 
 8   social_security_regime                 93285 non-null  int64 
 9   economic_activity_center               93285 non-null  int64 
 10  number_workers_center                  93285 non-null  int64 
 11  province_enterp

C:\Users\mique\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (53,54) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
variables.columns.values.tolist()

['accident_type',
 'gender',
 'nationality',
 'work_situation',
 'main_occupation',
 'seniority_in_the_job_months',
 'seniority_in_the_job_days',
 'contract_type',
 'social_security_regime',
 'economic_activity_center',
 'number_workers_center',
 'province_enterprise',
 'contract_subcontract',
 'ETT',
 'personal_assumption',
 'own_service',
 'foreign_prevention_service',
 'designated_workers',
 'joint_service',
 'none_preventive_organization',
 'accident_place',
 'traffic_accident',
 'center_belongs_worker_social_security',
 'center_where_worker_affiliated',
 'contract_subcontract_relationship',
 'ETT_relationship',
 'other_relations',
 'province',
 'workers',
 'economic_activity_type',
 'date_accident',
 'date',
 'date_medical_leave',
 'week_day',
 'time_day',
 'time_workday',
 'was_usual_job',
 'risk_evaluation',
 'place_type',
 'job_type',
 'specific_physical_activity',
 'material_physical_activity',
 'deviation',
 'Material_deviation',
 'contact_caused_injury',
 'material_contact',

In [3]:
variables=pd.read_csv('DATA/transport_accidents_2017_18.csv')
variables=variables[['gender','work_situation','main_occupation','seniority_in_the_job_months','age','province','workers','economic_activity_type','date_accident','week_day','time_workday','place_type','job_type','injury_severity','date']]
variables['age'] = variables['age'] #.replace('Desconegut', '404')

#categorizing age 

def categorize_age(age):
    if int(age) <= 25:
        return '16-25'
    elif int(age) <= 35 and int(age) > 25:
        return '26-35'
    elif int(age) <= 45 and int(age) > 35:
        return '36-45'
    elif int(age) <= 55 and int(age) > 45:
        return '46-55'
    elif age == '404':
        return 'Age not known'
    else:
        return '+56'

#specifying injuries from numbers to descriptions

def categorize_injury(injury_severity):
    if int(injury_severity) == 1:
        return 'mild'
    elif int(injury_severity) == 2:
        return 'serious'
    elif int(injury_severity) == 3:
        return 'very serious'
    elif int(injury_severity) == 4:
        return 'mortal'
    elif injury_severity == '404':
        return 'severity not known'
    
variables['age'] = variables['age'].apply(categorize_age)
variables['injury_severity'] = variables['injury_severity'].apply(categorize_injury)

x=variables['main_occupation'].unique()
print(x)
# count the frequency of each value in column 'a'
value_counts = variables['main_occupation'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

## mirar com pot ser que no tingui cap null

[841 843 833 844 834 842 832 831]
843    46.452270
841    21.349627
833    11.151846
844     9.576030
842     6.041700
832     2.815029
834     1.346412
831     1.267085
Name: main_occupation, dtype: float64


In [75]:
variables['Quarter'] = variables['date'].apply(lambda x: pd.Timestamp(x).quarter)
variables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93285 entries, 0 to 93284
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       93285 non-null  int64 
 1   work_situation               93285 non-null  int64 
 2   main_occupation              93285 non-null  int64 
 3   seniority_in_the_job_months  93285 non-null  int64 
 4   age                          93285 non-null  object
 5   province                     93285 non-null  int64 
 6   workers                      93285 non-null  int64 
 7   economic_activity_type       93285 non-null  int64 
 8   date_accident                93285 non-null  int64 
 9   week_day                     93285 non-null  int64 
 10  time_workday                 93285 non-null  int64 
 11  place_type                   93285 non-null  int64 
 12  job_type                     93285 non-null  int64 
 13  injury_severity              93

In [84]:

categorical = ['gender','main_occupation','seniority_in_the_job_months','age','province','workers','economic_activity_type','date_accident','week_day','time_workday','place_type','job_type','injury_severity','date']
import pandas as pd
import itertools 

#Create a list of diferent combinations of 3 variables of the given variables in categorical
combinations = list(itertools.combinations(categorical, 3))

# Load the dataset into a Pandas DataFrame
df = variables

#Create a dictonary that has the results for each combination of variables

dic={}

# Count the number of occurrences of each unique combination of values
for combination in combinations:
    counts = df.groupby(list(combination)).size().reset_index(name='count')
    counts['percentage %'] = counts['count'] / df.shape[0] * 100

    # Get the top 10 combinations of values that are the most repeated in the dataset
    top_counts = counts.sort_values('count', ascending=False).head(10)
    dic[tuple(combination)]=counts.sort_values('count', ascending=False)
dic3=dic
# It gives the combinations of values that are freqüent in more that 10 percent of all the accidents.
for key, value in dic.items():
    condition = value.iloc[:, -1] > 10
    result = value[condition]
    
    if not result.empty:
        print(result)
        print('-----------------------------------------------------')

    gender  main_occupation    age  count  percentage %
33       1              843  36-45  15330     16.433510
34       1              843  46-55  14470     15.511604
-----------------------------------------------------
     gender  main_occupation  economic_activity_type  count  percentage %
769       1              843                     494  22990     24.644905
-----------------------------------------------------
    gender  main_occupation  week_day  count  percentage %
42       1              843         1   9638     10.331779
-----------------------------------------------------
     gender  main_occupation  place_type  count  percentage %
229       1              843          13  17607     18.874417
-----------------------------------------------------
     gender  main_occupation  job_type  count  percentage %
168       1              843        12  14637     15.690626
194       1              843        61  10956     11.744653
----------------------------------------------

In [87]:
import pandas as pd
import itertools 

categorical = ['gender','main_occupation','seniority_in_the_job_months','age','province','workers','economic_activity_type','date_accident','week_day','time_workday','place_type','job_type','injury_severity','date']

#Create a list of diferent combinations of 4 variables of the given variables in categorical
combinations = list(itertools.combinations(categorical, 4))

# Load the dataset into a Pandas DataFrame
df = variables

#Create a dictonary of the top 10 counts for each combination

dic={}

# Count the number of occurrences of each unique combination of values
for combination in combinations:
    counts = df.groupby(list(combination)).size().reset_index(name='count')
    counts['percentage %'] = counts['count'] / df.shape[0] * 100

    # Get the top 10 combinations of values that are the most repeated in the dataset
    top_counts = counts.sort_values('count', ascending=False).head(10)
    dic[tuple(combination)]=counts.sort_values('count', ascending=False)
dic4=dic 
# It gives the combinations of 4 values that are freqüent in more that 5 percent of all the accidents.
for key, value in dic.items():
    condition = value.iloc[:, -1] > 5
    result = value[condition]
    
    if not result.empty:
        print(result)
        print('-----------------------------------------------------')

      gender  main_occupation    age  economic_activity_type  count  \
2336       1              843  36-45                     494   8239   
2510       1              843  46-55                     494   7943   

      percentage %  
2336      8.832074  
2510      8.514767  
-----------------------------------------------------
     gender  main_occupation    age  place_type  count  percentage %
902       1              843  36-45          13   6393      6.853192
949       1              843  46-55          13   6017      6.450126
-----------------------------------------------------
     gender  main_occupation    age  job_type  count  percentage %
754       1              843  36-45        12   5438      5.829447
785       1              843  46-55        12   4764      5.106930
-----------------------------------------------------
     gender  main_occupation    age injury_severity  count  percentage %
102       1              843  36-45            mild  15067     16.151578
106    

In [88]:
import pandas as pd
import itertools 

#Create a list of diferent combinations of 6 variables of the given variables in categorical
combinations = list(itertools.combinations(categorical, 6))

# Load the dataset into a Pandas DataFrame
df = variables

#Create a dictonary of the top 10 counts for each combination

dic={}

# Count the number of occurrences of each unique combination of values
for combination in combinations:
    counts = df.groupby(list(combination)).size().reset_index(name='count')
    counts['percentage %'] = counts['count'] / df.shape[0] * 100

    # Get the top 10 combinations of values that are the most repeated in the dataset
    top_counts = counts.sort_values('count', ascending=False).head(10)
    dic[tuple(combination)]=counts.sort_values('count', ascending=False)
dic6=dic   
# It gives the combinations of 6 values that are freqüent in more that 1 percent of all the accidents.
for key, value in dic.items():
    condition = value.iloc[:, -1] > 1
    result = value[condition]
    
    if not result.empty:
        print(result)
        print('-----------------------------------------------------')

      gender  main_occupation    age  economic_activity_type  week_day  \
7907       1              843  36-45                     494         1   
8669       1              843  46-55                     494         1   
7911       1              843  36-45                     494         2   
8672       1              843  46-55                     494         2   
7915       1              843  36-45                     494         3   
8676       1              843  46-55                     494         3   
7919       1              843  36-45                     494         4   
8684       1              843  46-55                     494         5   
7923       1              843  36-45                     494         5   
8680       1              843  46-55                     494         4   

     injury_severity  count  percentage %  
7907            mild   1820      1.951010  
8669            mild   1688      1.809508  
7911            mild   1521      1.630487  
8672     